In [2]:
import requests
import json
import sqlite3
from pprint import pprint
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

app_id = "130c281c"
app_key = "f5d8adb2213c2be776f6123bb77488ee"


base_url = "https://api.edamam.com/api/recipes/v2"

params = {
    'type':'public',
    'q':'chicken',
    'app_id': app_id,
    'app_key': app_key
}

# Get request
response = requests.get(base_url, params=params)
data = response.json()

pprint(data)

{'_links': {'next': {'href': 'https://api.edamam.com/api/recipes/v2?q=chicken&app_key=f5d8adb2213c2be776f6123bb77488ee&_cont=CHcVQBtNNQphDmgVQntAEX4BYldtAAIARGdBAGAbZFZ2BwYFUXlSCmpFNVEiAgQPRmxIBmsRagN0DVEEEDFCBTQUZVcgB1IVLnlSVSBMPkd5BgNK&type=public&app_id=130c281c',
                     'title': 'Next page'}},
 'count': 10000,
 'from': 1,
 'hits': [{'_links': {'self': {'href': 'https://api.edamam.com/api/recipes/v2/b79327d05b8e5b838ad6cfd9576b30b6?type=public&app_id=130c281c&app_key=f5d8adb2213c2be776f6123bb77488ee',
                               'title': 'Self'}},
           'recipe': {'calories': 4228.043058200812,
                      'cautions': ['Sulfites'],
                      'co2EmissionsClass': 'G',
                      'cuisineType': ['italian'],
                      'dietLabels': ['Low-Carb'],
                      'digest': [{'daily': 422.8934243674497,
                                  'hasRDI': True,
                                  'label': 'Fat',
               

In [3]:
print(data['hits'][0]['recipe']['cuisineType'])

['italian']


In [4]:
# attempting for loop to concatenate for chicken recipes

# Create IDs column for 200 records
created_ids = range(0,200)
recipe_id = (list(created_ids))

# For loop to create chicken recipe table
recipe_list = []
cuisine_type = []
total_calories = []
calories_per_serving = []
recipe_source = []
total_time = []
total_co2 = []
emission_class = []
number_of_ingredients = []
diet_labels = []
total_weight = []

for i in range(0,10):
    linkref = data['_links']['next']['href']
    response_data = requests.get(linkref)
    new_data = response_data.json()['hits']
    data = response_data.json()
    for i in new_data:
        names = i['recipe']['label']
        cuisines = i['recipe']['cuisineType']
        calories = i['recipe']['calories']
        source = i['recipe']['source']
        time = i['recipe']['totalTime']
        per_serving = i['recipe']['calories']/i['recipe']['yield']
        co2 = i['recipe']['totalCO2Emissions']
        classes = i['recipe']['co2EmissionsClass']
        ingredients = len(i['recipe']['ingredients'])
        diet = i['recipe']['dietLabels']
        weight = i['recipe']['totalWeight']        
        
        recipe_list.append(names)
        cuisine_type.append(cuisines)
        total_calories.append(calories)
        recipe_source.append(source)
        total_time.append(time)
        calories_per_serving.append(per_serving)
        total_co2.append(co2)
        emission_class.append(classes)
        number_of_ingredients.append(ingredients)
        diet_labels.append(diet)
        total_weight.append(weight)
        
    # Checking number of ingredients list to see if data was recorded
print(number_of_ingredients)

[2, 2, 6, 6, 6, 2, 12, 17, 10, 7, 11, 8, 3, 3, 17, 6, 10, 14, 9, 9, 9, 8, 8, 8, 8, 18, 7, 10, 3, 12, 6, 2, 11, 9, 10, 3, 5, 8, 5, 11, 6, 8, 15, 8, 4, 5, 8, 6, 5, 9, 10, 10, 9, 5, 9, 9, 14, 3, 12, 7, 13, 5, 14, 5, 7, 7, 10, 11, 11, 12, 14, 11, 10, 13, 6, 11, 10, 14, 18, 5, 13, 4, 9, 9, 10, 8, 6, 11, 10, 11, 6, 10, 5, 13, 10, 3, 5, 12, 13, 13, 6, 10, 17, 7, 12, 8, 4, 7, 6, 11, 17, 11, 12, 9, 9, 13, 6, 11, 5, 7, 10, 12, 11, 7, 11, 9, 11, 10, 6, 4, 9, 11, 10, 8, 10, 12, 4, 11, 5, 13, 10, 13, 12, 8, 13, 12, 8, 10, 8, 5, 11, 6, 6, 13, 14, 5, 10, 4, 10, 15, 7, 6, 13, 15, 6, 11, 8, 6, 8, 6, 7, 13, 5, 11, 12, 11, 4, 12, 12, 14, 10, 12, 5, 14, 12, 13, 13, 6, 10, 12, 10, 15, 4, 15, 6, 6, 7, 8, 6, 6]


In [5]:
chicken_recipe_df = pd.DataFrame({'recipe_id': recipe_id,
                                 'recipe_name': recipe_list,
                                 'cuisine_type': cuisine_type,
                                 'source': recipe_source,
                                 'total_time': total_time,
                                 'total_calories': total_calories,
                                 'calories_per_serving': calories_per_serving})
chicken_recipe_df.head(10)

,recipe_id,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving
0,0,Poached Chicken for Tacos,[mexican],Cookstr,120.0,432.480000,216.240000
1,1,Chicken Broth,[american],Cookstr,240.0,117.308431,58.654216
2,2,Fig & Balsamic Chicken,[french],Food52,60.0,1040.234000,346.744667
3,3,Basic Roast Chicken,[american],Cookstr,120.0,3797.291855,632.881976
4,4,Ultimate Roast Chicken,[american],My Recipes,0.0,2196.939412,366.156569
5,5,Chicken & peppers,[american],The Stone Soup,130.0,1123.850000,561.925000
6,6,Chicken Tamales,[south american],Food Network,120.0,16433.274364,684.719765
7,7,Chicken biryani,[indian],BBC,60.0,2057.673000,1028.836500
8,8,Roast Chicken With Chicken Livers,[french],Food & Wine,0.0,3800.558906,316.713242
9,9,Chicken Carbonara,[italian],Delish,25.0,2072.423000,518.105750


In [6]:
# Creating CO2 Emissions Dataframe
co2_df = pd.DataFrame({'recipe_id': recipe_id,
                      'total_co2': total_co2,
                      'emission_class': emission_class,
                      'diet_labels': diet_labels,
                      'total_weight': total_weight,
                      'number_of_ingredients': number_of_ingredients})
co2_df.head(10)

,recipe_id,total_co2,emission_class,diet_labels,total_weight,number_of_ingredients
0,0,4026.960000,F,"[High-Protein, Low-Carb, Low-Sodium]",408.000000,2
1,1,13435.534076,G,"[Low-Carb, Low-Sodium]",54.783084,2
2,2,6575.985636,F,"[Low-Fat, Low-Sodium]",879.622222,6
3,3,27074.433030,G,[Low-Carb],2852.308290,6
4,4,13597.746146,F,[Low-Carb],1409.710267,6
5,5,7037.275000,G,[Low-Carb],947.500000,2
6,6,37443.976996,F,"[Balanced, High-Fiber]",4720.335568,12
7,7,7043.271562,G,[High-Fiber],1401.547222,17
8,8,19270.308929,F,[Low-Carb],2082.162893,10
9,9,5479.003138,F,[],1136.900000,7


In [7]:
chicken_recipe_df.to_csv('chicken_recipe.csv', index=False)

co2_df.to_csv('co2.csv', index=False)

In [8]:
chicken_df = dd.read_csv('chicken_recipe.csv')
co2_df = dd.read_csv('co2.csv')


In [9]:
co2_df.head()

,recipe_id,total_co2,emission_class,diet_labels,total_weight,number_of_ingredients
0,0,4026.960000,F,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,2
1,1,13435.534076,G,"['Low-Carb', 'Low-Sodium']",54.783084,2
2,2,6575.985636,F,"['Low-Fat', 'Low-Sodium']",879.622222,6
3,3,27074.433030,G,['Low-Carb'],2852.308290,6
4,4,13597.746146,F,['Low-Carb'],1409.710267,6


In [10]:
chicken_df.head()

,recipe_id,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving
0,0,Poached Chicken for Tacos,['mexican'],Cookstr,120.0,432.480000,216.240000
1,1,Chicken Broth,['american'],Cookstr,240.0,117.308431,58.654216
2,2,Fig & Balsamic Chicken,['french'],Food52,60.0,1040.234000,346.744667
3,3,Basic Roast Chicken,['american'],Cookstr,120.0,3797.291855,632.881976
4,4,Ultimate Roast Chicken,['american'],My Recipes,0.0,2196.939412,366.156569


In [11]:
merged_df = dd.merge(chicken_df, co2_df, left_index=True, right_index=True)


In [14]:
merged_df.head(3)

,recipe_id_x,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving,recipe_id_y,total_co2,emission_class,diet_labels,total_weight,number_of_ingredients
0,0,Poached Chicken for Tacos,['mexican'],Cookstr,120.0,432.480000,216.240000,0,4026.960000,F,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,2
1,1,Chicken Broth,['american'],Cookstr,240.0,117.308431,58.654216,1,13435.534076,G,"['Low-Carb', 'Low-Sodium']",54.783084,2
2,2,Fig & Balsamic Chicken,['french'],Food52,60.0,1040.234000,346.744667,2,6575.985636,F,"['Low-Fat', 'Low-Sodium']",879.622222,6


In [47]:
cleaned_df = merged_df.drop(['recipe_id_x', 'recipe_id_y', 'source', 'emission_class', 'number_of_ingredients',], axis=1)
cleaned_df.head(3)

,recipe_name,cuisine_type,total_time,total_calories,calories_per_serving,total_co2,diet_labels,total_weight
0,Poached Chicken for Tacos,['mexican'],120.0,432.480000,216.240000,4026.960000,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000
1,Chicken Broth,['american'],240.0,117.308431,58.654216,13435.534076,"['Low-Carb', 'Low-Sodium']",54.783084
2,Fig & Balsamic Chicken,['french'],60.0,1040.234000,346.744667,6575.985636,"['Low-Fat', 'Low-Sodium']",879.622222


In [49]:
cleaned_df['Total_CO2_Per_Weight'] = (cleaned_df['total_co2']/cleaned_df['total_weight'])
cleaned_df.head()

,recipe_name,cuisine_type,total_time,total_calories,calories_per_serving,total_co2,diet_labels,total_weight,Total_CO2_Per_Weight
0,Poached Chicken for Tacos,['mexican'],120.0,432.480000,216.240000,4026.960000,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,9.870000
1,Chicken Broth,['american'],240.0,117.308431,58.654216,13435.534076,"['Low-Carb', 'Low-Sodium']",54.783084,245.249683
2,Fig & Balsamic Chicken,['french'],60.0,1040.234000,346.744667,6575.985636,"['Low-Fat', 'Low-Sodium']",879.622222,7.475920
3,Basic Roast Chicken,['american'],120.0,3797.291855,632.881976,27074.433030,['Low-Carb'],2852.308290,9.492113
4,Ultimate Roast Chicken,['american'],0.0,2196.939412,366.156569,13597.746146,['Low-Carb'],1409.710267,9.645774


In [30]:
cleaned_df.groupby('recipe_name').total_co2.mean().compute()

recipe_name
Apricot Glazed Chicken                      5854.449735
Asian Grilled Chicken                      15042.841404
Balsamic Chicken                            4619.096871
Barbecue Chicken                           16800.357088
Barbecued Chicken                          37415.700620
                                               ...     
The Smith's Chicken-Fried Chicken Skins     7532.415217
Tuscan Chicken                             64975.389624
Twistin’ Chicken                            7741.530451
Ultimate Roast Chicken                     13597.746146
Un-Fried Chicken                           18357.268000
Name: total_co2, Length: 154, dtype: float64

In [50]:
cleaned_df.groupby('recipe_name').Total_CO2_Per_Weight.mean().compute()

recipe_name
Apricot Glazed Chicken                      6.412389
Asian Grilled Chicken                       9.627881
Balsamic Chicken                            9.240645
Barbecue Chicken                            8.479005
Barbecued Chicken                           8.047561
                                             ...    
The Smith's Chicken-Fried Chicken Skins     5.174324
Tuscan Chicken                             24.904872
Twistin’ Chicken                            8.244898
Ultimate Roast Chicken                      9.645774
Un-Fried Chicken                            7.193544
Name: Total_CO2_Per_Weight, Length: 154, dtype: float64

In [57]:
recipe_analysis = cleaned_df.groupby('recipe_name').agg({'total_co2': ['mean', 'std'], 'total_weight': ['mean', 'count']}).compute()
recipe_analysis

total_co2              \
                                                 mean         std   
recipe_name                                                         
Poached Chicken for Tacos                 4026.960000         NaN   
Chicken Broth                            20374.395391  9813.03178   
Fig & Balsamic Chicken                    6575.985636         NaN   
Basic Roast Chicken                      27074.433030         NaN   
Ultimate Roast Chicken                   13597.746146         NaN   
...                                               ...         ...   
The Smith's Chicken-Fried Chicken Skins   7532.415217         NaN   
Saucy chicken & vegetables                9824.902654         NaN   
Chicken Tortilla Soup                    12014.121638         NaN   
Chicken and Tortellini Soup              19460.936237         NaN   
Skillet Chicken and Rice                  9944.733747         NaN   

                                        total_weight        
                                                mean count  
recipe_name                                                 
Poached Chicken for Tacos                 408.000000     1  
Chicken Broth                              92.736182     2  
Fig & Balsamic Chicken                    879.622222     1  
Basic Roast Chicken                      2852.308290     1  
Ultimate Roast Chicken                   1409.710267     1  
...                                              ...   ...  
The Smith's Chicken-Fried Chicken Skins  1455.729249     1  
Saucy chicken & vegetables               1345.710341     1  
Chicken Tortilla Soup                    2231.996784     1  
Chicken and Tortellini Soup              1895.589533     1  
Skillet Chicken and Rice                 1442.532833     1  

[154 rows x 4 columns]